In [1]:
import requests

In [8]:
base_url = "https://musicbrainz.org/ws/2"
user_agent = "AlbumMetadataTest/1.0 (testing@example.com)"
rate_limit_delay = 1.0  # 1 second between requests
url = f"{base_url}/release/"
query = "Aqualung"
params = {
    'query': query,#f"{query} AND format:Vinyl",
    'fmt': 'json',
    'limit': rate_limit_delay
}
headers = {
    'User-Agent': user_agent
}

In [35]:
response = requests.get(url, params=params, headers=headers)
data = response.json()

data = [media for media in data['releases'] if 'Vinyl' in media['media'][0]['format']]

In [37]:
len(data)

8

In [13]:
data.keys()

dict_keys(['created', 'count', 'offset', 'releases'])

In [39]:
data

[{'id': '221eb7c4-ca11-44f9-b83a-81c73fd0b77b',
  'score': 100,
  'status-id': '4e304316-386d-3409-af2e-78857eec5cfe',
  'packaging-id': 'f7101ce3-0384-39ce-9fde-fbbd0044d35f',
  'artist-credit-id': 'e4f6d118-e75b-3ccd-acca-9cad5c90045e',
  'count': 1,
  'title': 'Aqualung',
  'status': 'Official',
  'disambiguation': '200g',
  'packaging': 'Cardboard/Paper Sleeve',
  'text-representation': {'language': 'eng', 'script': 'Latn'},
  'artist-credit': [{'name': 'Jethro Tull',
    'artist': {'id': 'ece57992-dc2e-4f67-a269-fa43626c1a3d',
     'name': 'Jethro Tull',
     'sort-name': 'Jethro Tull'}}],
  'release-group': {'id': '204cc47e-2e1a-3877-8e1e-d14bfffd19aa',
   'type-id': 'f529b476-6e62-324f-b0aa-1f3e33d313fc',
   'primary-type-id': 'f529b476-6e62-324f-b0aa-1f3e33d313fc',
   'title': 'Aqualung',
   'primary-type': 'Album'},
  'date': '2007',
  'country': 'US',
  'release-events': [{'date': '2007',
    'area': {'id': '489ce91b-6658-3307-9877-795b68554c98',
     'name': 'United States',

In [ ]:

class MusicBrainzAPI:
    """MusicBrainz API client for album metadata"""
    
    def __init__(self):
        self.base_url = "https://musicbrainz.org/ws/2"
        self.user_agent = "AlbumMetadataTest/1.0 (testing@example.com)"
        self.rate_limit_delay = 1.0  # 1 second between requests
        
    def search_albums(self, query: str, limit: int = 10) -> Dict:
        """Search for albums using MusicBrainz API"""
        url = f"{self.base_url}/release/"
        params = {
            'query': query,
            'fmt': 'json',
            'limit': limit
        }
        headers = {
            'User-Agent': self.user_agent
        }
        
        try:
            print(f"🎵 Searching MusicBrainz for: '{query}'...")
            response = requests.get(url, params=params, headers=headers)
            response.raise_for_status()
            
            # Rate limiting
            time.sleep(self.rate_limit_delay)
            
            data = response.json()
            return {
                'success': True,
                'data': data,
                'total_results': data.get('count', 0),
                'results': data.get('releases', [])
            }

In [ ]:
import requests
import json
from pprint import pprint

# Your Last.fm API key
API_KEY = "4823bcda7e5085e60c69ec1baae5c8b0"
BASE_URL = "https://ws.audioscrobbler.com/2.0/"

def get_similar_artists(artist_name, limit=20):
    """
    Fetch similar artists for a given artist
    """
    params = {
        'method': 'artist.getsimilar',
        'artist': artist_name,
        'api_key': API_KEY,
        'format': 'json',
        'limit': limit
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        return None

def get_artist_info(artist_name, mbid=None):
    """
    Fetch detailed artist information including tags/genres
    """
    params = {
        'method': 'artist.getinfo',
        'api_key': API_KEY,
        'format': 'json'
    }

    # Prefer MBID if available, otherwise use artist name
    if mbid:
        params['mbid'] = mbid
        print(f"🎵 Fetching with MBID: {mbid}")
    else:
        params['artist'] = artist_name
        print(f"🎵 Fetching with name: {artist_name}")

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        return None

def display_similar_artists(data):
    """
    Pretty print similar artists data
    """
    if not data or 'similarartists' not in data:
        print("No similar artists found")
        return

    artists = data['similarartists']['artist']
    print(f"\n{'='*70}")
    print(f"Found {len(artists)} similar artists:")
    print(f"{'='*70}\n")

    for i, artist in enumerate(artists[:10], 1):  # Show first 10
        mbid = artist.get('mbid', 'N/A')
        match = float(artist.get('match', 0))
        print(f"{i:2}. {artist['name']:<30} | Match: {match:.3f} | MBID: {mbid[:8] if mbid != 'N/A' else 'N/A'}...")

def display_artist_info(data):
    """
    Pretty print artist info including genres/tags
    """
    if not data or 'artist' not in data:
        print("No artist info found")
        return

    artist = data['artist']
    print(f"\n{'='*70}")
    print(f"Artist: {artist['name']}")
    print(f"{'='*70}")

    # Stats
    if 'stats' in artist:
        print(f"\n📊 Statistics:")
        print(f"   Listeners: {int(artist['stats'].get('listeners', 0)):,}")
        print(f"   Playcount: {int(artist['stats'].get('playcount', 0)):,}")

    # Tags/Genres
    if 'tags' in artist and 'tag' in artist['tags']:
        tags = artist['tags']['tag']
        print(f"\n🏷️  Genres/Tags ({len(tags)} total):")
        for i, tag in enumerate(tags[:5], 1):  # Show first 5
            print(f"   {i}. {tag['name']}")

    # MBID
    if artist.get('mbid'):
        print(f"\n🆔 MusicBrainz ID: {artist['mbid']}")

    # Bio (first 200 chars)
    if 'bio' in artist and 'summary' in artist['bio']:
        bio = artist['bio']['summary']
        print(f"\n📝 Bio: {bio[:200]}...")

def main():
    print("=" * 70)
    print("Last.fm API Test Script")
    print("=" * 70)

    # Test 1: Get similar artists
    print("\n\n### TEST 1: Get Similar Artists ###\n")
    artist_name = "The Beatles"
    print(f"Searching for artists similar to: {artist_name}")

    similar_data = get_similar_artists(artist_name, limit=20)
    if similar_data:
        display_similar_artists(similar_data)

        # Save to file for inspection
        with open('similar_artists.json', 'w') as f:
            json.dump(similar_data, f, indent=2)
        print("\n✅ Full response saved to: similar_artists.json")

    # Test 2: Get artist info by name
    print("\n\n### TEST 2: Get Artist Info (by name) ###\n")
    artist_name = "Wishbone Ash"
    print(f"Fetching info for: {artist_name}")

    info_data = get_artist_info(artist_name)
    if info_data:
        display_artist_info(info_data)

        # Save to file
        with open('artist_info_name.json', 'w') as f:
            json.dump(info_data, f, indent=2)
        print("\n✅ Full response saved to: artist_info_name.json")

    # Test 3: Get artist info by MBID (more accurate)
    print("\n\n### TEST 3: Get Artist Info (by MBID) ###\n")

    # First, get MBID from similar artists
    if similar_data and 'similarartists' in similar_data:
        first_similar = similar_data['similarartists']['artist'][0]
        artist_name = first_similar['name']
        artist_mbid = first_similar.get('mbid')

        if artist_mbid:
            print(f"Fetching info for: {artist_name}")
            print(f"Using MBID: {artist_mbid}")

            info_data_mbid = get_artist_info(artist_name, mbid=artist_mbid)
            if info_data_mbid:
                display_artist_info(info_data_mbid)

                # Save to file
                with open('artist_info_mbid.json', 'w') as f:
                    json.dump(info_data_mbid, f, indent=2)
                print("\n✅ Full response saved to: artist_info_mbid.json")
        else:
            print(f"❌ No MBID available for {artist_name}")

    print("\n" + "="*70)
    print("Tests complete!")
    print("="*70)

if __name__ == "__main__":
    # Check if API key is set
    if API_KEY == "YOUR_LASTFM_API_KEY_HERE":
        print("⚠️  Please set your Last.fm API key in the script!")
        print("Get one at: https://www.last.fm/api/account/create")
    else:
        main()